**Biomedical Software Engineering**

**Prof. Arthur Goldberg**

**Dept. Genetics and Genomic Sciences**

**Spring 1, 2020**

# Class composition and inheritance

## Composition

In [27]:
# composition of sexes, subjects and samples
from enum import Enum

class Sex(Enum):
    female = 1
    male = 2
    unknown = 3

class Subject(object):
    """An individual human in a study"""
    def __init__(self, id, gender):
        self.id = id
        self.gender = gender

    def __str__(self):
        return f"id: {self.id}; gender: {self.gender.name}"

subject_1 = Subject(23, Sex.female)
print(subject_1)

id: 23; gender: female


Now we create `class Sample`, which reuses `Subject` via composition.

In [28]:
class Sample(object):
    """A set of subjects in a study"""
    def __init__(self, name):
        self.id = name
        self.subjects = {}

    def add(self, subject):
        # todo: check that a subject with subject.id is not already in this Sample
        self.subjects[subject.id] = subject
        
    def get(self, id):
        # todo: useful error if subject not in this Sample
        return self.subjects[id]
        
    def count(self):
        """Number of subjects"""
        return len(self.subjects)

subject_2 = Subject(78, Sex.male)
sample = Sample('example_sample')
print('sample.count():', sample.count())
sample.add(subject_1)
sample.add(subject_2)
print('sample.count():', sample.count())
print('sample.get(78)', sample.get(78))

sample.count(): 0
sample.count(): 2
sample.get(78) id: 78; gender: male


## Inheritance, and its benefits
Inheritance reuses existing class(es). It enables hierarchical refinement and extension.

The existing class that's reused is the *base* or *superclass* class, and the new class is the *derived* or *subclass* class. An instance of a dervied class can access the attributes (data and methods) of its base class. It may also override them. The [`super()`](https://docs.python.org/3/library/functions.html#super) function accesses methods and attributes in the base class.

All classes inherit from Python's built-in `object` class, the root of the class derivation DAG.

In [29]:
class Root(object):
    " manages id and name "
    def __init__(self, id, name):
        self.id = id
        self.name = name
    def get_id(self):
        return self.id
    def get_name(self):
        return self.name

class DataElement(Root):
    " manage a data element "
    def __init__(self, id, name, data):
        self.data = data
        super().__init__(id, name)

from math import pi
data_element = DataElement(1, 'pi', pi)
# get id and name from the base class
print('id:', data_element.get_id())
print(f'name: {data_element.get_name()}')
print(f'data: {data_element.data:.6f}')

id: 1
name: pi
data: 3.141593


In [30]:
class AnnotatedDataElement(DataElement):
    " annotate a data element "
    def __init__(self, id, name, data, annotation):
        self.annotation = annotation
        super().__init__(id, name, data)

from math import e
annotated_data_element = AnnotatedDataElement(2, 'e', e, 'a great constant')
ade = annotated_data_element
print(ade.id, ade.name, ade.data, "'{}'".format(ade.annotation))

# illustrate the inheritance hierarchy
for cls in [object, Root, DataElement, AnnotatedDataElement, float]:
    print("annotated_data_element in an instance of {}: {}".format(cls.__name__,
          isinstance(annotated_data_element, cls)))

2 e 2.718281828459045 'a great constant'
annotated_data_element in an instance of object: True
annotated_data_element in an instance of Root: True
annotated_data_element in an instance of DataElement: True
annotated_data_element in an instance of AnnotatedDataElement: True
annotated_data_element in an instance of float: False


### A practical example of inheritance
[DE Sim](https://github.com/KarrLab/de_sim) is an object oriented [discrete event simulator](https://en.wikipedia.org/wiki/Discrete-event_simulation) I've built. Our multialgorithmic whole-cell simulator, [WC Sim](https://github.com/KarrLab/wc_sim), uses DE Sim.

A discrete event simulation (DES) models a system as a sequence of discrete events that occur at increasing instants in time. An OO DES structures a simulation as a set of simulation objects, each of which models an element of the simulation. For example, a simulation of air travel would have simulation objects representing airport air traffic control centers and airplanes. Objects schedule an event by sending an event message that specifies the time and nature of the event to model. Objects model the behavior of their entity by executing their events in increasing (actually non-decreasing) time order.

To enable easy construction of simulations, DE Sim defines a generic simulation object `ApplicationSimulationObject` and a generic simulation message `SimulationMessage`.
The code below defines `TemplatePeriodicSimulationObject`, a simulation object derived from `ApplicationSimulationObject` that executes periodically. Note that `TemplatePeriodicSimulationObject` cannot be used directly, but must be subclassed by an object that overrides `handle_event(self)` with a method that actually handles events.


In [0]:
class Error(Exception):
    """ Base class for exceptions in de_sim

    Attributes:
        message (:obj:`str`): the exception's message
    """
    def __init__(self, message=None):
        super().__init__(message)

# SimulatorError is derived from Error which is derived from Exception, a Python convention
class SimulatorError(Error):
    """ Exception raised for errors in de_sim

    Attributes:
        message (:obj:`str`): the exception's message
    """
    def __init__(self, message=None):
        super().__init__(message)

# These 3 class definitions are stubs
class SimulationMessage(object):
  pass

class UniformSequence(object):
  pass

class ApplicationSimulationObject(object):
  pass

class NextEvent(SimulationMessage):
    pass

class TemplatePeriodicSimulationObject(ApplicationSimulationObject):
    """ Template self-clocking ApplicationSimulationObject

    Events occur at time 0, `period`, `2 x period`, ...

    Attributes:
        period (:obj:`float`): interval between events, in simulated seconds
        event_time_sequence (:obj:`UniformSequence`): a uniform sequence generator
    """
    def __init__(self, name, period):
        if period <= 0:
            raise SimulatorError("period must be positive, but is {}".format(period))
        self.period = period
        self.event_time_sequence = UniformSequence(0, period)
        super().__init__(name)

    def schedule_next_event(self):
        """ Schedule the next event at `self.period` simulated time in the future
        """
        next_event_time = self.event_time_sequence.__next__()
        # Uses `send_event_absolute` in `ApplicationSimulationObject`
        self.send_event_absolute(next_event_time, self, NextEvent())

    def handle_event(self):
        """ Handle the periodic event

        Derived classes must override this method and actually handle the event
        """
        pass    # pragma: no cover     # must be overridden

    def send_initial_events(self):
        # create the initial event
        self.schedule_next_event()

    def handle_simulation_event(self, event):
        self.handle_event()
        self.schedule_next_event()


### Abstract base classes
Python [abstract base classes](https://docs.python.org/3/library/abc.html) (ABCs) are classes used to define properties shared by multiple subclasses (see [tutorial](https://pymotw.com/3/abc/)). ABCs achieve this by serving as shared nodes in the inheritance DAG:

In [32]:
from abc import ABC

class MyABC(ABC): pass

class MyClass(MyABC): pass

assert issubclass(MyClass, MyABC)
assert isinstance(MyClass(), MyABC)

# an int is not a subclass of MyABC, of course
assert isinstance(3, MyABC)

AssertionError: ignored

An ABC defines a partially enforced interface. An ABC that defines a set of methods, but not their bodies, forces subclasses to implement the methods.

ABCs cannot be instantiated themselves. Instead, they're used as superclasses of subclasses that can be instantiated.

In [33]:
from abc import ABC, abstractmethod

class C(ABC):
    # @abstractmethod decorates abstract methods
    @abstractmethod
    def my_abstract_method(self, arg1):
        pass

    @classmethod
    @abstractmethod
    def my_abstract_classmethod(cls):
        pass

    @staticmethod
    @abstractmethod
    def my_abstract_staticmethod(arg2):
        pass

class D(C):

    CONSTANT = 37
    def __init__(self, data):
        self.data = data

    def my_abstract_method(self, arg1):
        print(self.data, arg1)

    @classmethod
    def my_abstract_classmethod(cls):
        print('CONSTANT =', cls.CONSTANT)

    @staticmethod
    def my_abstract_staticmethod(arg3):
        print('all i know is', arg3)

d = D('my name')
d.my_abstract_method('is mud')
d.my_abstract_classmethod()
d.my_abstract_staticmethod(38)

my name is mud
CONSTANT = 37
all i know is 38


Subclasses of an ABC must implement the ABC's methods.

In [34]:
class Bad(C):

    def __init__(self, data):
        self.data = data

    def my_abstract_method(self, arg1):
        print(self.data, arg1)

    @classmethod
    def my_abstract_classmethod(cls):
        pass

Bad(1)

TypeError: ignored

But adherence to method signatures is optional.

In [35]:
class OK(C):

    def __init__(self, data):
        self.data = data

    # note the extra argument
    def my_abstract_method(self, arg1, extra_arg):
        print(self.data, arg1)

    @classmethod
    def my_abstract_classmethod(cls): pass

    @staticmethod
    def my_abstract_staticmethod(arg3): pass

ok = OK(1)
ok.my_abstract_method(2, 3)

1 2


ABCs are used by Python to define `Sequence`, `MutableSequence`, and the numeric types hierarchy (see [PEP 3141](https://www.python.org/dev/peps/pep-3141)).